<a href="https://colab.research.google.com/github/kumarsh8483/AI/blob/master/sparknlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark==3.1.2
!pip install sparknlp
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212.4 MB 58 kB/s 
     |████████████████████████████████| 198 kB 65.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880769 sha256=db5a652bb320274dbe10ac1efce64873b6703644a5e43565bf04fa9f65878591
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 4.9 MB/s 


In [ ]:
spark = sparknlp.start()

In [ ]:
import pandas as pd
df = pd.read_csv('/content/input_data.csv')
#df.values.tolist()
df

,Unnamed: 0,reqnumber,li_orderid,cde_name,E,F,li_cat,H,li_name,J,K,L,req_name,li_description
0,1,A102,9901,1000,Cat,1,Cat,9901,John LLC4,1,1,1,Smith,equipment for cleaning
1,2,A103,9901,1000,Cat,1,Cat,9901,John LLC5,1,1,1,Smith,bottles for storage
2,3,A104,9901,1000,Cat,2,Cat,9901,John LLC6,1,1,1,Smith,mopping stick
3,4,A105,9901,1000,Cat,3,Cat,9901,John LLC7,1,1,1,Smith,cleaner
4,5,A106,9901,1000,Cat,4,Cat,9901,John LLC8,1,1,1,Smith,hammers and wrenches
5,6,A107,9901,1000,Cat,5,Cat,9901,John LLC9,1,1,1,Smith,pliers
6,7,A108,9901,1000,Cat,6,Cat,9901,John LLC10,1,1,1,Smith,tool set


In [ ]:

from pyspark.sql.types import StringType
data = spark.createDataFrame (df['li_description'].values.tolist(),StringType()).toDF("text")
data.show(truncate=80)

+----------------------+
|                  text|
+----------------------+
|equipment for cleaning|
|   bottles for storage|
|         mopping stick|
|               cleaner|
|  hammers and wrenches|
|                pliers|
|              tool set|
+----------------------+



In [173]:

from sparknlp.base import *
from sparknlp.annotator import *

documentAssembler = DocumentAssembler()\
    .setInputCol("textcol")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

stopwords = StopWordsCleaner()\
    .setInputCols("token")\
    .setOutputCol("stopwords")
           

normalizer = Normalizer() \
    .setInputCols(["stopwords"]) \
    .setOutputCol("normalized")\
    .setLowercase(True)\
    .setCleanupPatterns(["[^\w\d\s]"]) # remove punctuations (keep alphanumeric chars)
    # if we don't set CleanupPatterns, it will only keep alphabet letters ([^A-Za-z])

nlpPipeline = Pipeline().setStages([
    documentAssembler, 
    tokenizer,
    stopwords,
    normalizer
 ])


#empty_df = spark.createDataFrame(data).toDF("text")
pipelineModel = nlpPipeline.fit(data)

In [174]:
result = pipelineModel.transform(data)
result.show()
df3=result.select('normalized.result').show(truncate=False)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|           stopwords|          normalized|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|equipment for cle...|[{document, 0, 21...|[{token, 0, 8, eq...|[{token, 0, 8, eq...|[{token, 0, 8, eq...|
| bottles for storage|[{document, 0, 18...|[{token, 0, 6, bo...|[{token, 0, 6, bo...|[{token, 0, 6, bo...|
|       mopping stick|[{document, 0, 12...|[{token, 0, 6, mo...|[{token, 0, 6, mo...|[{token, 0, 6, mo...|
|             cleaner|[{document, 0, 6,...|[{token, 0, 6, cl...|[{token, 0, 6, cl...|[{token, 0, 6, cl...|
|hammers and wrenches|[{document, 0, 19...|[{token, 0, 6, ha...|[{token, 0, 6, ha...|[{token, 0, 6, ha...|
|              pliers|[{document, 0, 5,...|[{token, 0, 5, pl...|[{token, 0, 5, pl...|[{token, 0, 5, pl...|
|            tool set|[{document, 0, 

In [162]:
df2 = pd.read_excel('/content/codes_ref.xlsx',header=2)
df2

data1 = spark.createDataFrame (df2['Description'].values.tolist(),StringType()).toDF("text1")
data1.show(truncate=80)


+-------------------------------------+
|                                text1|
+-------------------------------------+
|                  pliers and wrenches|
|mops, buckets, and signs for cleaning|
|                  chlorine 10 gallons|
|                       something else|
|        another description I made up|
+-------------------------------------+



In [169]:
from sparknlp.base import *
from sparknlp.annotator import *

documentAssembler1 = DocumentAssembler()\
    .setInputCol("text1")\
    .setOutputCol("document1")

tokenizer1 = Tokenizer() \
    .setInputCols(["document1"]) \
    .setOutputCol("token1")

stopwords1 = StopWordsCleaner()\
    .setInputCols("token1")\
    .setOutputCol("stopwords1")
           

normalizer1 = Normalizer() \
    .setInputCols(["stopwords1"]) \
    .setOutputCol("normalized1")\
    .setLowercase(True)\
    .setCleanupPatterns(["[^\w\d\s]"]) # remove punctuations (keep alphanumeric chars)
    # if we don't set CleanupPatterns, it will only keep alphabet letters ([^A-Za-z])

nlpPipeline = Pipeline().setStages([
    documentAssembler, 
    tokenizer,
    stopwords,
    normalizer
 ])


#empty_df = spark.createDataFrame(data).toDF("text")
pipelineModel1 = nlpPipeline.fit(data1)

In [192]:
result1 = pipelineModel1.transform(data1)
result1.show()
df4=result1.select('normalized.result').show(truncate=False)
df4 


+--------------------+--------------------+--------------------+--------------------+--------------------+
|               text1|            document|               token|           stopwords|          normalized|
+--------------------+--------------------+--------------------+--------------------+--------------------+
| pliers and wrenches|[{document, 0, 18...|[{token, 0, 5, pl...|[{token, 0, 5, pl...|[{token, 0, 5, pl...|
|mops, buckets, an...|[{document, 0, 36...|[{token, 0, 3, mo...|[{token, 0, 3, mo...|[{token, 0, 3, mo...|
| chlorine 10 gallons|[{document, 0, 18...|[{token, 0, 7, ch...|[{token, 0, 7, ch...|[{token, 0, 7, ch...|
|      something else|[{document, 0, 13...|[{token, 0, 8, so...|[{token, 0, 8, so...|[{token, 0, 8, so...|
|another descripti...|[{document, 0, 28...|[{token, 0, 6, an...|[{token, 0, 6, an...|[{token, 0, 6, an...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

+--------------------------------+
|

In [ ]:
from jellyfish import jaro_winkler_similarity
from similarity import jaro_winkler

jarowinkler = jaro_winkler_similarity
df["jarowinkler_sim"] = [jarowinkler.similarity(i,j) for i,j in zip(result[f"normalized"].reshape(1,-1),result1[f"normalized"].reshape(1,-1))]